<a href="https://colab.research.google.com/github/YufeiM28/CAPP_Static_Project/blob/main/Milestone2_VisualCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install --upgrade altair==5.4.1 typing_extensions==4.12.2 vl-convert-python
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
import altair as alt
import pandas as pd
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
df_1112 = pd.read_csv("1112_clean.csv")
df_1819 = pd.read_csv("1819_clean.csv")
df_2324 = pd.read_csv("2324_clean.csv")

# Plot 1

In [4]:
for df in (df_1112, df_1819, df_2324):
    df.columns = df.columns.str.strip()
    df = df.replace("NDA", None)

for df in (df_1112, df_1819, df_2324):
    df["College_Enrollment_Rate_School"] = pd.to_numeric(df["College_Enrollment_Rate_School"], errors="coerce")
    df["Graduation_Rate_School"] = pd.to_numeric(df["Graduation_Rate_School"], errors="coerce")

mean_df = pd.DataFrame({
    "Academic_Year": ["2011–12", "2018–19", "2023–24"],
    "College Enrollment Rate": [
        df_1112["College_Enrollment_Rate_School"].mean(),
        df_1819["College_Enrollment_Rate_School"].mean(),
        df_2324["College_Enrollment_Rate_School"].mean()],
    "Graduation Rate": [
        df_1112["Graduation_Rate_School"].mean(),
        df_1819["Graduation_Rate_School"].mean(),
        df_2324["Graduation_Rate_School"].mean()]
    })

In [5]:
# plot
plot_df = mean_df.melt("Academic_Year", var_name="Rate Type", value_name="Average Rate")

chart = (
    alt.Chart(plot_df)
    .mark_bar()
    .encode(
        x=alt.X("Academic_Year:N", title="Academic Year", axis=alt.Axis(labelAngle=0)),
        xOffset=alt.XOffset("Rate Type:N"),
        y=alt.Y("Average Rate:Q", title="Average Rate (%)", axis=alt.Axis(grid=False)),
        color=alt.Color("Rate Type:N", title="Outcome Type",
              scale=alt.Scale(domain=["College Enrollment Rate", "Graduation Rate"],
              range=["#0072B2", "#009E73"]),
              legend=alt.Legend(titleFontSize=13, labelFontSize=12)))
    .properties(
        width=560, height=380,
        title=alt.TitleParams(
        text="Trends in High School Graduation and College Enrollment Rates (2011–2024)",
        fontSize=16)))

labels = (alt.Chart(plot_df)
    .mark_text(dy=-6, fontSize=11)
    .encode(
        x="Academic_Year:N",
        xOffset="Rate Type:N",
        y="Average Rate:Q",
        text=alt.Text("Average Rate:Q", format=".1f"),
        detail="Rate Type:N"))

final_chart = (chart + labels).configure_view(strokeWidth=0)

final_chart.save("plot1.svg")

final_chart

/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(
/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(


alt.LayerChart(...)

# Plot 2

In [6]:
# data clean
df_1112["School_Year"] = "2011-12"
df_1819["School_Year"] = "2018-19"
df_2324["School_Year"] = "2023-24"

dfs = [df_1112, df_1819, df_2324]

for df in dfs:
    df.dropna(subset=["Student_Attendance_Avg", "Teacher_Attendance_Avg"], inplace=True)

merged_wide = pd.concat(dfs, ignore_index=True)
merged_wide.to_csv("attendance_merged_wide.csv", index=False)

merged_long = merged_wide.melt(
    id_vars=["School_ID", "School_Year"],
    value_vars=["Student_Attendance_Avg", "Teacher_Attendance_Avg"],
    var_name="Role", value_name="Attendance"
    ).replace({"Role": {"Student_Attendance_Avg": "Student",
    "Teacher_Attendance_Avg": "Teacher"}})

merged_long["Attendance"] = pd.to_numeric(merged_long["Attendance"], errors="coerce")
merged_long = merged_long.dropna(subset=["Attendance", "School_Year", "Role"])

year_labels = ["2011-12", "2018-19", "2023-24"]

In [18]:
# ChatGPT is used for helping me to learn how to generate jitterplot in Altair
plot2 = (
    alt.Chart(merged_long)
    .transform_calculate(
        xIndex=f"indexof({year_labels!r}, datum.School_Year)",
        jitter="(random() - 0.5) * 0.45",
        xj="datum.xIndex + datum.jitter")
    .mark_circle(size=60, opacity=0.7)
    .encode(
        x=alt.X("xj:Q", axis=alt.Axis(
                title="School Year",
                grid = False,
                values=[0, 1, 2],
                labelExpr=f"{year_labels!r}[datum.value]")),
        y=alt.Y("Attendance:Q", title="Attendance (%)", scale=alt.Scale(domain=[0, 100]),
                axis=alt.Axis(titleFontSize=13, labelFontSize=12)),
        color=alt.Color("Role:N",
            scale=alt.Scale(domain=["Student", "Teacher"], range=["#99cdee", "#ee9999"]),
            legend=alt.Legend(titleFontSize=13, labelFontSize=12)))
    .properties(width=560, height=380,
            title=alt.TitleParams(
            text="Student and Teacher Attendance across School Year", fontSize=16)))

plot2.save("plot2.svg")

plot2

/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(
/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(


alt.Chart(...)

# Plot 3 & 4

In [8]:
# clean data
dfs = [df_1112[["School_ID", "School_Survey_Safety", "School_Year"]],
       df_1819[["School_ID", "School_Survey_Safety", "School_Year"]],
       df_2324[["School_ID", "School_Survey_Safety", "School_Year"]]]

safety = pd.concat(dfs, ignore_index=True).copy()

safety["School_Survey_Safety"] = (safety["School_Survey_Safety"].astype(str).str.strip().str.upper())

order6 = ["VERY WEAK", "WEAK", "NEUTRAL", "STRONG", "VERY STRONG", "NOT ENOUGH DATA"]

safety["Safety6"] = pd.Categorical(
    safety["School_Survey_Safety"].replace({"VERYSTRONG":"VERY STRONG"}),
    categories=order6,
    ordered=True)

safety = (safety
          .dropna(subset=["School_ID", "School_Year"])
          .dropna(subset=["Safety6"], how="all"))

In [9]:
# OptionA
safety = safety.copy()
safety["Safety6"] = (
    safety["Safety6"]
    .astype("string")
    .str.strip()
    .str.upper()
)
safety["Safety6_clean"] = safety["Safety6"].fillna("NOT ENOUGH DATA")

# color
order6 = ["VERY WEAK", "WEAK", "NEUTRAL", "STRONG", "VERY STRONG"]
colors6 = ["#a50f15", "#de2d26", "#fdae6b", "#fdd0a2", "#fee5d9"]

heat = (
    alt.Chart(safety)
      .mark_rect()
      .encode(
          x=alt.X("School_ID:N", title="Schools", axis=alt.Axis(labels=False, ticks=False)),
          y=alt.Y("School_Year:N", title="Academic Year", sort=["2011-12","2018-19","2023-24"]),
          color=alt.Color(
              "Safety6_clean:N",
              title="Safety Level",
              scale=alt.Scale(domain=order6, range=colors6),
              legend=alt.Legend(orient="right")))
      .properties(width=560, height=200,
        title=alt.TitleParams(text="School Safety Level Across Years", fontSize=16)))

heat.save("plot3.svg")

heat

/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(
/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(


alt.Chart(...)

In [10]:
# OptionB
levels5 = ["VERY WEAK", "WEAK", "NEUTRAL", "STRONG", "VERY STRONG"]
year_order = ["2011-12","2018-19","2023-24"]

safety_5 = safety.loc[safety["Safety6_clean"].isin(levels5)].copy()

counts = (safety_5.groupby(["School_Year", "Safety6_clean"])
            .size()
            .reset_index(name="count"))

grid = (pd.MultiIndex.from_product([year_order, levels5],
                                   names=["School_Year", "Safety6_clean"]).to_frame(index=False))
counts_full = (grid.merge(counts, on=["School_Year","Safety6_clean"], how="left").fillna({"count": 0}))

# white to red
red_range = ["#ffffff", "#fee5d9", "#fcae91", "#fb6a4a", "#cb181d"]

heat_density = (
    alt.Chart(counts_full)
      .mark_rect()
      .encode(x=alt.X("Safety6_clean:N",
                  title="Safety Level",
                  sort=levels5,
                  axis=alt.Axis(labelAngle=0, labelFontSize=12, titleFontSize=13)),
          y=alt.Y("School_Year:N", title="School Year", sort=year_order),
          color=alt.Color("count:Q", title="School Density", scale=alt.Scale(range=red_range)))
      .properties(title="School Safety Heatmap across Years", width=560, height=200))

heat_density.save("plot4.svg")

heat_density

/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(
/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(


alt.Chart(...)

# Plot 5

In [11]:
# clean
def coerce_numeric_series(s):
    s = s.astype(str).str.strip().str.replace("%", "", regex=False)
    s = pd.to_numeric(s, errors="coerce")
    return s

def prep(df, year_label):
    use = df[["School_ID", "SAT_Grade_11_Score_School_Avg", "College_Enrollment_Rate_School"]].copy()
    use["SAT_Grade_11_Score_School_Avg"] = coerce_numeric_series(use["SAT_Grade_11_Score_School_Avg"])
    use["College_Enrollment_Rate_School"] = coerce_numeric_series(use["College_Enrollment_Rate_School"])
    use["School_Year"] = year_label
    return use.dropna(subset=["SAT_Grade_11_Score_School_Avg", "College_Enrollment_Rate_School"])

df_1819_sc = prep(df_1819, "2018-19")
df_2324_sc = prep(df_2324, "2023-24")

scatter_df = pd.concat([df_1819_sc, df_2324_sc], ignore_index=True)


year_order = ["2018-19", "2023-24"]
colors = ["#1f78b4", "#33a02c"]

In [12]:
# plot
plot5 = (
    alt.Chart(scatter_df)
      .mark_circle(size=65, opacity=0.75)
      .encode(
          x=alt.X(
              "SAT_Grade_11_Score_School_Avg:Q",
              title="SAT Grade 11 – School Average",
              scale=alt.Scale(domain=[600, 1600])),
          y=alt.Y(
              "College_Enrollment_Rate_School:Q",
              title="College Enrollment Rate (%)",
              scale=alt.Scale(domain=[0, 100])),
          color=alt.Color(
              "School_Year:N",
              title="School Year",
              scale=alt.Scale(domain=year_order, range=colors)))
      .properties(title="SAT vs College Enrollment Rate by School", width=560, height=300))

plot5.save("plot5.svg")

plot5

/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(
/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(


alt.Chart(...)

# Plot 6

In [13]:
# clean data
cols_scores = ["Family_Involvement_Score", "Environment Score", "Instruction Score"]
keep = ["School_ID", "Student_Attendance_Avg"] + cols_scores

d = df_1112[keep].copy()

for c in cols_scores + ["Student_Attendance_Avg"]:
    d[c] = pd.to_numeric(d[c], errors="coerce")

long = (d.melt(id_vars=["School_ID", "Student_Attendance_Avg"],
           value_vars=cols_scores,
           var_name="Domain", value_name="Score")
     .dropna(subset=["Score", "Student_Attendance_Avg"]))

# labels
long["Domain"] = long["Domain"].map({
    "Family_Involvement_Score": "Family Involvement",
    "Environment Score": "Environment",
    "Instruction Score": "Instruction"})

# color
domain = ["Family Involvement", "Environment", "Instruction"]
palette = ["#1b9e77", "#80b1d3", "#fb9a99"]

# plot
scatter = (alt.Chart(long)
      .mark_circle(size=60, opacity=0.7)
      .encode(
          x=alt.X("Score:Q", title="Evaluation Score",
                  scale=alt.Scale(domain=[0, 100])),
          y=alt.Y("Student_Attendance_Avg:Q", title="Student Attendance (%)",
                  scale=alt.Scale(domain=[60, 100])),
          color=alt.Color("Domain:N", title="Score",
                          scale=alt.Scale(domain=domain, range=palette))
      ))

# fit lines
trend = (alt.Chart(long)
      .transform_regression("Score", "Student_Attendance_Avg", groupby=["Domain"])
      .mark_line(size=2)
      .encode(x="Score:Q", y="Student_Attendance_Avg:Q",
          color=alt.Color("Domain:N", scale=alt.Scale(domain=domain, range=palette))))

plot6 = (scatter + trend).properties(
    width=560,
    height=380,
    title=alt.TitleParams(text="Influential Factors towards Students' Attendance Rate", fontSize=16))

plot6.save("plot6.svg")

plot6

/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(
/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(


alt.LayerChart(...)

# Plot 7 & 8

In [14]:
### OptionA
# clean data & build dataframe
def prep_creative(df, year_label):
    d = df[["School_ID", "Creative_School_Certification"]].copy()
    d["Creative_School_Certification"] = (
        d["Creative_School_Certification"]
        .astype(str)
        .str.strip()
        .str.upper())
    d = d[~d["Creative_School_Certification"].isin(["NDA", "NOT ENOUGH DATA", "INCOMPELET DATA"])]
    d = d.dropna(subset=["Creative_School_Certification"])
    d["School_Year"] = year_label
    return d

df_1819_ldr = prep_creative(df_1819, "2018-19")
df_2324_ldr = prep_creative(df_2324, "2023-24")

combined = pd.concat([df_1819_ldr, df_2324_ldr], ignore_index=True)

summary = (combined
    .groupby(["School_Year", "Creative_School_Certification"])
    .size()
    .reset_index(name="Count"))

summary["Pct"] = summary.groupby("School_Year")["Count"].transform(lambda x: x / x.sum() * 100)

level_order = ["EMERGING", "DEVELOPING", "STRONG", "EXCELLING"]
summary = summary[summary["Creative_School_Certification"].isin(level_order)]

In [15]:
# plot
line = (alt.Chart(summary)
      .transform_filter(alt.FieldOneOfPredicate(field="Creative_School_Certification", oneOf=level_order))
      .encode(
          x=alt.X("School_Year:N", sort=year_order, title=None, axis=alt.Axis(labelAngle=0)),
          y=alt.Y("Pct:Q", title="Percentage of Schools (%)"),
          color=alt.Color("Creative_School_Certification:N",
                          title="Creativity Level",
                          sort=level_order,
                          scale=alt.Scale(range=["#b8e3b0","#78c679","#31a354","#006d2c"])),
          detail="Creative_School_Certification:N"))

line_chart = (line.mark_line(point=True, strokeWidth=2)
    .properties(title="Creativity Shift across Years", width=420, height=360))

line_chart.save("plot7.svg")

line_chart

/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(
/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(


alt.Chart(...)

In [16]:
### Option B
# The stacked figure is brainstormed by ChatGPT.
# I provided my previous option to ChatGPT and asked if there is any other way to visualize the same data.
# I would adopt stacked bars if I only use current datasets without adding other longitudinal data.

# clean data
missing = {"NDA", "NOT ENOUGH DATA", "INCOMPLETE DATA"}
level_order = ["EXCELLING", "STRONG", "DEVELOPING", "EMERGING"]
combined = combined[combined["Creative_School_Certification"].isin(level_order)].copy()
rank_map = {"EMERGING":1, "DEVELOPING":2, "STRONG":3, "EXCELLING":4}
combined["level_rank"] = combined["Creative_School_Certification"].map(rank_map)

year_order = ["2018-19", "2023-24"]

In [17]:
# plot
stack = (alt.Chart(combined)
      .mark_bar()
      .encode(
          x=alt.X("School_Year:N", sort=year_order, title=None, axis=alt.Axis(labelAngle=0)),
          y=alt.Y("count():Q", stack="normalize", axis=alt.Axis(format="%", title="Share of Schools")),
          color=alt.Color("Creative_School_Certification:N",
              title="Creativity Certification",
              sort=["EMERGING", "DEVELOPING", "STRONG", "EXCELLING"],
              scale=alt.Scale(range=["#b8e3b0", "#78c679", "#31a354", "#006d2c"])),
          order=alt.Order("level_rank:Q", sort="descending"))
      .properties(
          title="Share of Schools by Creativity Level", width=400, height=200)
      .configure_view(strokeWidth=0)
      .configure_axis(grid=False))

stack.save("plot8.svg")

stack

/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(
/usr/local/lib/python3.12/dist-packages/altair/utils/data.py:71: UserWarning: You passed a `<class 'narwhals.stable.v1.DataFrame'>` to `is_pandas_dataframe`.

Hint: Instead of e.g. `is_pandas_dataframe(df)`, did you mean `is_pandas_dataframe(df.to_native())`?
  return _is_pandas_dataframe(obj) or isinstance(


alt.Chart(...)